In [30]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from time import sleep
import os
import pandas as pd

pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 50)
pd.options.display.float_format = '{:,.4f}'.format

## Scrape Top WBTC-WETH Uniswap V3 Pools from revert finance

We now scrape this table of [Top Pools](https://revert.finance/#/top-positions?sort=underlying_value&underlying-value-from=20000&apr-from=20&token1=0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2&token0=0x2260fac5e5542a773aa44fbcfedf7c193bc2c599&age-from=14) and put the info in a data frame.

In [2]:
# set up
chrome_options = Options()
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('start-maximized')
chrome_options.add_argument("disable-infobars")
s = Service('/Applications/chromedriver')

In [3]:
browser = webdriver.Chrome(service=s, options = chrome_options)
url = 'https://revert.finance/#/top-positions?sort=underlying_value&underlying-value-from=20000&apr-from=20&token1=0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2&token0=0x2260fac5e5542a773aa44fbcfedf7c193bc2c599&age-from=14'
browser.get(url)
delay = 20 #secods
WebDriverWait(browser, delay)
sleep(delay) # need to wait until the page is fully loaded before scraping

# scrape page and parse
html = browser.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
soup = BeautifulSoup(html, "html.parser")

# close browser
browser.close() 

In [4]:
# extract table header
header1 = [item.text for item in soup.find_all("span", class_='text-gray-20')]
header2 = [item_n.text for item_n in soup.find_all('a', class_='text-gray-20 visited:text-gray-20 hover:text-white active:gray-40')]
header = [header1[0]] + header2[:3] + [header1[1]] + header2[3:]

In [5]:
# extract pool/fees column
pool_fees = [item.text for item in soup.find_all("span", class_='text-gray-30')]

# extract nft_id/owner column
pool_links = ['https://revert.finance'+item.get('href') for item in soup.find_all('a', class_="underline text-green-40 visited:text-green-40 hover:text-green-20 active:green-60")]

In [6]:
# extract PnL, APR, fee APR, value columns
ha = [item.text for item in soup.find_all('div', class_="text-ellipsis overflow-hidden whitespace-nowrap")]

pnls = []
aprs = []
fee_aprs = []
vals = []
for i in range(len(ha)):
    j = i%4
    if j==0:
        pnls.append(ha[i])
    elif j==1:
        aprs.append(ha[i])
    elif j==2:
        fee_aprs.append(ha[i])
    elif j==3:
        vals.append(ha[i])
    else:
        pass

In [7]:
# extract age column
age_days = [item.text for item in soup.find_all('div', class_="border-b border-gray-70 transition-all ease-linear delay-30 duration-150 text-sm h-[60px] group-hover:bg-gray-90 flex items-center group-last:border-transparent group-hover:border-b-gray-70 justify-end pl-4 w-32 text-gray-30")]

In [25]:
# scrape the price range for each of these top pools 
price_rngs = []
for url in pool_links:
    browser = webdriver.Chrome(service=s, options = chrome_options)
    browser.get(url)
    delay = 30 #secods, increase this number if throws error
    WebDriverWait(browser, delay)
    sleep(delay) # need to wait until the page is fully loaded before scraping

    # scrape page and parse
    html = browser.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
    soup = BeautifulSoup(html, "html.parser")

    # close browser
    browser.close() 
    
    # extract price range
    price_range = [item.text for item in soup.find_all('div', class_='lp-assets') if item.text.__contains__('WBTC/WETH')][0].split(' WBTC/WETH')[0]
    price_rngs.append(price_range)

In [26]:
df = pd.DataFrame({'pool/fees':pool_fees, 'nft_id/owner':pool_links, 'PnL': pnls, 
                   'APR':aprs, 'fee APR':fee_aprs, 'value':vals, 'age':age_days, 'price_range':price_rngs,})
df

,pool/fees,nft_id/owner,PnL,APR,fee APR,value,age,price_range
0,0.30%,https://revert.finance/#/uniswap-position/main...,"$85,084.14",20.30%,20.39%,"$2,804,470.28",54.5 days,11.5061 - 15.1631
1,0.30%,https://revert.finance/#/uniswap-position/main...,"$142,183.59",67.14%,37.43%,"$822,076.02",76.1 days,12.1445 - 13.6928
2,0.30%,https://revert.finance/#/uniswap-position/main...,"$40,051.53",27.46%,27.54%,"$814,482.37",69.5 days,13.6108 - 15.9086
3,0.30%,https://revert.finance/#/uniswap-position/main...,"$37,183.85",23.96%,23.99%,"$756,293.96",74.9 days,13.0511 - 15.3461
4,0.30%,https://revert.finance/#/uniswap-position/main...,"$162,142.64",20.68%,20.95%,"$731,351.51",460.3 days,12.1445 - 16.1975
5,0.30%,https://revert.finance/#/uniswap-position/main...,"$33,903.50",23.86%,23.96%,"$692,087.80",74.9 days,13.0511 - 15.3461
6,0.30%,https://revert.finance/#/uniswap-position/main...,"$12,674.21",87.56%,91.74%,"$657,723.03",15.2 days,12.3650 - 14.5394
7,0.30%,https://revert.finance/#/uniswap-position/main...,"$19,732.69",20.82%,20.96%,"$629,284.50",55.0 days,11.7151 - 15.6248
8,0.30%,https://revert.finance/#/uniswap-position/main...,"$18,400.48",20.65%,21.43%,"$463,959.90",87.0 days,11.3008 - 15.2543
9,0.30%,https://revert.finance/#/uniswap-position/main...,"$22,634.59",20.63%,25.99%,"$356,716.93",110.4 days,12.0718 - 15.9086


## Clean the data and save as csv

In [27]:
# clean data
df['PnL'] = df.PnL.str.strip('$').str.replace(',', '').astype(float)
df['APR'] = df.APR.str.strip('%').astype(float)
df['fee APR'] = df['fee APR'].str.strip('%').astype(float)
df['value'] = df.value.str.strip('$').str.replace(',', '').astype(float)
df['age'] = df.age.str.replace(' days', '').astype(float)
df['price_lwr'], df['price_upr'] = zip(*df.price_range.str.split(' - '))

In [28]:
df = df.rename(columns={'nft_id/owner':'url'})
df.columns = df.columns.str.replace('/| ', '_', regex=True)
df = df.drop(columns='price_range')
df

,pool_fees,url,PnL,APR,fee_APR,value,age,price_lwr,price_upr
0,0.30%,https://revert.finance/#/uniswap-position/main...,"85,084.1400",20.3000,20.3900,"2,804,470.2800",54.5000,11.5061,15.1631
1,0.30%,https://revert.finance/#/uniswap-position/main...,"142,183.5900",67.1400,37.4300,"822,076.0200",76.1000,12.1445,13.6928
2,0.30%,https://revert.finance/#/uniswap-position/main...,"40,051.5300",27.4600,27.5400,"814,482.3700",69.5000,13.6108,15.9086
3,0.30%,https://revert.finance/#/uniswap-position/main...,"37,183.8500",23.9600,23.9900,"756,293.9600",74.9000,13.0511,15.3461
4,0.30%,https://revert.finance/#/uniswap-position/main...,"162,142.6400",20.6800,20.9500,"731,351.5100",460.3000,12.1445,16.1975
5,0.30%,https://revert.finance/#/uniswap-position/main...,"33,903.5000",23.8600,23.9600,"692,087.8000",74.9000,13.0511,15.3461
6,0.30%,https://revert.finance/#/uniswap-position/main...,"12,674.2100",87.5600,91.7400,"657,723.0300",15.2000,12.3650,14.5394
7,0.30%,https://revert.finance/#/uniswap-position/main...,"19,732.6900",20.8200,20.9600,"629,284.5000",55.0000,11.7151,15.6248
8,0.30%,https://revert.finance/#/uniswap-position/main...,"18,400.4800",20.6500,21.4300,"463,959.9000",87.0000,11.3008,15.2543
9,0.30%,https://revert.finance/#/uniswap-position/main...,"22,634.5900",20.6300,25.9900,"356,716.9300",110.4000,12.0718,15.9086


In [31]:
os.makedirs('../data', exit_ok=True)
df.to_csv('../data/univ3_wbtc_weth_lp_top_positions.csv')